In [1]:
installed.packages()

,Package,LibPath,Version,Priority,Depends,Imports,LinkingTo,Suggests,Enhances,License,License_is_FOSS,License_restricts_use,OS_type,MD5sum,NeedsCompilation,Built
abind,abind,/usr/local/lib/R/site-library,1.4-5,NA,R (>= 1.5.0),"methods, utils",NA,NA,NA,LGPL (>= 2),NA,NA,NA,NA,no,3.6.3
acepack,acepack,/usr/local/lib/R/site-library,1.4.1,NA,NA,NA,NA,testthat,NA,MIT + file LICENSE,NA,NA,NA,NA,yes,3.6.3
actuar,actuar,/usr/local/lib/R/site-library,3.0-0,NA,R (>= 3.3.0),"stats, graphics, expint",expint,MASS,NA,GPL (>= 2),NA,NA,NA,NA,yes,3.6.3
ade4,ade4,/usr/local/lib/R/site-library,1.7-15,NA,R (>= 2.10),"graphics, grDevices, methods, stats, utils, MASS, pixmap, sp",NA,"ade4TkGUI, adegraphics, adephylo, ape, CircStats, deldir, lattice, spdep, splancs, waveslim",NA,GPL (>= 2),NA,NA,NA,NA,yes,3.6.3
AER,AER,/usr/local/lib/R/site-library,1.2-9,NA,"R (>= 3.0.0), car (>= 2.0-19), lmtest, sandwich (>= 2.4-0), survival (>= 2.37-5), zoo","stats, Formula (>= 0.2-0)",NA,"boot, dynlm, effects, fGarch, forecast, foreign, ineq, KernSmooth, lattice, longmemo, MASS, mlogit, nlme, nnet, np, plm, pscl, quantreg, rgl, ROCR, rugarch, sampleSelection, scatterplot3d, strucchange, systemfit (>= 1.1-20), truncreg, tseries, urca, vars",NA,GPL-2 | GPL-3,NA,NA,NA,NA,no,3.6.3
akima,akima,/usr/local/lib/R/site-library,0.6-2.1,NA,R (>= 2.0.0),sp,NA,NA,NA,ACM | file LICENSE,NA,yes,NA,NA,yes,3.6.3
alabama,alabama,/usr/local/lib/R/site-library,2015.3-1,NA,"R (>= 2.10.1), numDeriv",NA,NA,NA,NA,GPL (>= 2),NA,NA,NA,NA,no,3.6.3
annotate,annotate,/usr/local/lib/R/site-library,1.64.0,NA,"R (>= 2.10), AnnotationDbi (>= 1.27.5), XML","Biobase, DBI, xtable, graphics, utils, stats, methods, BiocGenerics (>= 0.13.8), RCurl",NA,"hgu95av2.db, genefilter, Biostrings (>= 2.25.10), IRanges, rae230a.db, rae230aprobe, tkWidgets, GO.db, org.Hs.eg.db, org.Mm.eg.db, hom.Hs.inp.db, humanCHRLOC, Rgraphviz, RUnit,",NA,Artistic-2.0,NA,NA,NA,NA,no,3.6.3
AnnotationDbi,AnnotationDbi,/usr/local/lib/R/site-library,1.48.0,NA,"R (>= 2.7.0), methods, utils, stats4, BiocGenerics (>= 0.29.2), Biobase (>= 1.17.0), IRanges","DBI, RSQLite, S4Vectors (>= 0.9.25)",NA,"hgu95av2.db, GO.db, org.Sc.sgd.db, org.At.tair.db, KEGG.db, RUnit, TxDb.Hsapiens.UCSC.hg19.knownGene, hom.Hs.inp.db, org.Hs.eg.db, reactome.db, AnnotationForge, graph, EnsDb.Hsapiens.v75, BiocStyle, knitr",NA,Artistic-2.0,NA,NA,NA,NA,no,3.6.3
ape,ape,/usr/local/lib/R/site-library,5.4,NA,R (>= 3.2.0),"nlme, lattice, graphics, methods, stats, tools, utils, parallel, Rcpp (>= 0.12.0)",Rcpp,"gee, expm, igraph",NA,GPL-2 | GPL-3,NA,NA,NA,NA,yes,3.6.3


In [1]:
library(Seurat)
library(edgeR)

Loading required package: limma



In [2]:
main_dir <- "/projects/robson-lab/research/endometriosis"
sample_id <-"Endometriosis-revision-3"
data_dir <- "DEG/edgeR-input"
out_dir <- "DEG/edgeR-output"
setwd(main_dir)

In [3]:
subtypes_list <-  (read.csv(paste("analysis",sample_id,data_dir,"subtypelist.csv",sep="/"), row.names=1))
#subtypes <- c(subtypes_list$X0)

In [4]:
for (types in subtypes_list$X0){
    print (paste("analyzing",types))
    raw_counts <- read.csv(paste("analysis",sample_id,data_dir,paste(types,"counts.csv",sep="-"),sep="/"), row.names=1)
    metadata <- read.csv(paste("analysis",sample_id,data_dir,paste(types,"metadata.csv",sep="-"), sep="/"), row.names=1)
    rownames(metadata) <- gsub("-", ".", rownames(metadata))
    groups <- factor(metadata$group)
    cds <- DGEList(counts = raw_counts, group = groups,genes = raw_counts$index)
    
    nc <- cpm(cds, normalized.lib.sizes=FALSE)
    design <- model.matrix(~0 + groups)
    cds <- estimateDisp(cds, design, robust=TRUE)
    fit <- glmQLFit(cds, design,robust=TRUE)
    if ("EcO" %in% groups == TRUE){
        con <- makeContrasts(eueVSctrl = groupsEuE - groupsCtrl,
                             ecpVSctrl = groupsEcP - groupsCtrl,
                             ecpaVSctrl = groupsEcPA - groupsCtrl,
                             ecoVSctrl = groupsEcO - groupsCtrl,
                             ecpVSeco = groupsEcP - groupsEcO,
                             ecpaVSeco = groupsEcPA - groupsEcO, levels = design)
        } else {
        con <- makeContrasts(eueVSctrl = groupsEuE - groupsCtrl,
                             ecpVSctrl = groupsEcP - groupsCtrl,
                             ecpaVSctrl = groupsEcPA - groupsCtrl,levels = design)
        }
    #de.tgw <- exactTest(cds,design)
    anov <- glmQLFTest(fit, contrast=con)
    
    diff.dat <- topTags(anov,n=nrow(cds$counts))
    res <- diff.dat$table
    file <- paste("analysis/",sample_id,out_dir, paste(types,"DEG.csv",sep="-"),sep="/")
    #file2 <- paste("analysis/",sample_id,out_dir, paste(types,"DEG-top200.csv",sep="-"),sep="/")
    write.csv(res[res$FDR < 0.01,],file=file)
    #write.csv((row.names(res))[0:200],file=file2)
}

[1] "analyzing glandular early-secretory"
[1] "analyzing mid-secretory"
[1] "analyzing glandular"
[1] "analyzing lumenal 2"
[1] "analyzing lumenal 1"
[1] "analyzing mesothelial"
[1] "analyzing lumenal"
[1] "analyzing MUC5B+"
[1] "analyzing ciliated"


--- end of process ---